In [ ]:
%run optimizers/Optimizer.ipynb

class ShapeEstimation(Optimizer):
    
    def __init__(self, mean, pcaBasis, pcaVariance):
        Optimizer.__init__(self, mean, pcaBasis, pcaVariance)
        self.allowedDeviationMultiple = 3
        self.pcaStdv = np.sqrt(self.pcaVariance)
    
    def open(self):
        self.session = tf.Session()
        self._initializeVariables()
        self._setError()
        self._setOptimizer()
        model = tf.global_variables_initializer()
        self.session.run(model)
    
    def close(self):
        self.session.close()
    
    def optimize(self, targetFeatures):
        self._setCComponent()
        self._setHComponent(targetFeatures)
        for _ in range(10):
            for _ in range(50):
                self.session.run(self.optimizer)            
            self._constrainA()
        for _ in range(5):
            self.session.run(self.optimizer) 
        return (np.array(self.session.run(self.a)),
                self.r,
                self.t,
                self.s)
    
    def update(self, a, r, t, s):
        self.session.run(self.a.assign(a))
        self.r = r
        self.t = t
        self.s = s
        
    def _initializeVariables(self):
        self.C = tf.Variable(np.zeros((2*self.numFeatures, self.numComponents)), name='C', trainable=False)
        self.h = tf.Variable(np.zeros((2*self.numFeatures, 1)), name='h', trainable=False)
        self.a = tf.Variable(np.zeros((self.numComponents, 1)), name='a', trainable=True)
        self.allowedDeviationMax = tf.constant(self.allowedDeviationMultiple * self.pcaStdv, dtype=tf.float64)
        self.allowedDeviationMin = tf.constant(-1 * self.allowedDeviationMultiple * self.pcaStdv, dtype=tf.float64)
        ai, ri, ti, si = Optimizer.getDefaults(self.numComponents)
        self.r = ri
        self.t = ti
        self.s = si
    
    def _setCComponent(self):
        C = np.zeros((2*self.numFeatures, self.numComponents))
        for i in range(self.numFeatures):
            C[2*i] = (self.r[0,0] * self.pcaBasis[3*i,:]
                    + self.r[0,1] * self.pcaBasis[3*i+1,:]
                    + self.r[0,2] * self.pcaBasis[3*i+2,:])
            C[2*i+1] = (self.r[1,0] * self.pcaBasis[3*i,:]
                    + self.r[1,1] * self.pcaBasis[3*i+1,:]
                    + self.r[1,2] * self.pcaBasis[3*i+2,:])
        C *= self.s
        self.session.run(self.C.assign(C))
        
    def _setHComponent(self, targetFeatures):
        h = np.zeros((2*self.numFeatures, 1))
        for i in range(self.numFeatures):
            h[2*i]   = targetFeatures[2*i] - self.s * (np.matmul(self.r[0], self.mean[3*i:3*i+3]) + self.t[0,0])
            h[2*i+1] = targetFeatures[2*i+1] - self.s * (np.matmul(self.r[1], self.mean[3*i:3*i+3]) + self.t[1,0])
        self.session.run(self.h.assign(h))
    
    def _setError(self):
        self.error = tf.losses.mean_squared_error(
            labels = tf.matmul(self.C, self.a),
            predictions = self.h
        )
        self.error = tf.reduce_sum(tf.square(self.h - tf.matmul(self.C, self.a)))
        
    def _setOptimizer(self):
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(self.error)
    
    def _constrainA(self):
        '''
        a = np.array(self.session.run(self.a))
        for i in range(self.numComponents):
            allowedDeviation = self.allowedDeviationMultiple * self.pcaStdv[i][0]
            if a[i][0] > allowedDeviation:
                a[i][0] = allowedDeviation
            elif a[i][0] < -allowedDeviation:
                a[i][0] = -allowedDeviation
        '''
        self.session.run(self.a.assign(tf.clip_by_value(self.a, self.allowedDeviationMin, self.allowedDeviationMax)))
        